#### How to run this
You can either start the jupyter server via the Opentrons-App under Robot > Settings > Advanced Settings.

Or you can ssh into the robots raspberry pie navigate to /var/lib/jupyter/notebooks (I have done this) and then you can probably start a jupyter notebook from there. To ssh into the robot you would need the IP and if you use the wired IP (can be found in the Opentrons-APP) you need to look that up again every time you connect as it changes. You could also try go via the WLAN (different IP, probalby permenent). 

However I usually found it easiest to go via the APP

In [1]:
# if you run this notebook on the robot and want to execute the commands you put in, use the following import
#from opentrons.execute import get_protocol_api

# if you are not running this on the robot and just want to simulate what the robot would do, use the following import
from opentrons.simulate import get_protocol_api

In [2]:
# the following command is executed in the terminal to stop the robot server to give this protocol contorl over gpios
!systemctl stop opentrons-robot-server

# if the above command is not executed the protocol will not have access to gpios like lights etc.
protocol = get_protocol_api("2.13")

# send the arm home
protocol.home()

Der Befehl "systemctl" ist entweder falsch geschrieben oder
konnte nicht gefunden werden.
C:\Users\mxwur\.opentrons\robot_settings.json not found. Loading defaults
C:\Users\mxwur\.opentrons\deck_calibration.json not found. Loading defaults


### Labware
All predefined labware can be found at at the [labware library](https://labware.opentrons.com/).
Otherwise we might need to do a custom labware definition which should be doable.

The current definitions are:
* _culture_wellplate_: 96-Wellplate on deckslot 2 containing the 230ul of medium inoculated with a bacteria culture. 
This is currently a [corning_96_wellplate_360ul_flat](https://labware.opentrons.com/corning_96_wellplate_360ul_flat?category=wellPlate). I understand that you want a non-flat bottom but this is the only wellplate I found that is neither a pcr nor a deep wellplate. You might want to choose a different wellplate.
* _target_wellplate_: 96-Wellplate on deckslot 1 that is initially empty. Again this is currently a [corning_96_wellplate_360ul_flat](https://labware.opentrons.com/corning_96_wellplate_360ul_flat?category=wellPlate), you might want to change that.
* _glycerol_wellplate_: 96-Deep-Wellplate on deckslot 4 containing glycerol. This is currently a [opentrons_96_deep_well_adapter_nest_wellplate_2ml_deep](https://labware.opentrons.com/opentrons_96_deep_well_adapter_nest_wellplate_2ml_deep), a V-bottom with 2ml volume per well. This might have to be adjusted aswell.
* _tiprack_300ul_1_: a tiprack with 96 300ul no filter tips on deckslot 3. See [detail](https://labware.opentrons.com/opentrons_96_tiprack_300ul?category=tipRack).
* _tiprack_300ul_2_: Same as above on deckslot 5. Only needed at all if you are using all 96 wells. Only ever needs to contain 1 tip in upper left slot. This is the additional tip you need for dispensing the glycerol in step 1 if you need 96 tips in step 2 (96 wells used -> 96 culture transfers in step 2 -> 96 tips needed for step 2)

### Steps
1. transfer 115ul from _glycerol_wellplate_ to each well in _target_wellplate_.
2. transfer 115ul from each well in _culture_wellplate_ to each well in _target_wellplate_.
3. mix content of _target_wellplate_ by aspirateing and dispensing into the same well a few times.

To save tips steps 2 and 3 are performed in paralell, meaning after each transfer from a _culture_wellplate_ to a _target_wellplate_ the pipette will mix the content of the _target_wellplate_ by aspirating and dispensing a few times.

In [3]:
# load hardware
culture_wellplate = protocol.load_labware('corning_96_wellplate_360ul_flat', 2)
target_wellplate = protocol.load_labware('corning_96_wellplate_360ul_flat', 1)
glycerol_wellplate = protocol.load_labware('opentrons_96_deep_well_adapter_nest_wellplate_2ml_deep', 4)

tiprack_300ul_1 = protocol.load_labware('opentrons_96_tiprack_300ul', 3)

#only needed if you use all 96 wells of the cultere/target wellplates
tiprack_300ul_2 = protocol.load_labware('opentrons_96_tiprack_300ul', 5)


# bigger pipette (30 - 300 µL)
p300 = protocol.load_instrument('p300_single', mount='right', tip_racks=[tiprack_300ul_1, tiprack_300ul_2]) 


The smaller pipette (1 - 10 µL) is not required for this protocol but if you want to load it anyway use the following code and do not forget to place the 20ul tiprack on the appropriate deckslot (here 5)
```(python)
tiprack_20ul = protocol.load_labware('opentrons_96_tiprack_20ul', 5)
p10 = protocol.load_instrument('p10_single', mount='left', tip_racks=[tiprack_20ul])
```

In [4]:
# rows and columns of the in the 96 wellplates
# if you use only a subset of these, cut the row or col indices you do not use from the lists
rows = [chr(x) for x in range(ord("A"), ord("H")+1)] #letters from A to H
cols = list(range(12))

# list of relevant wells in the 96 wellplates
# use col+1 as named wellindexing stats with 1 (first well is "A1" not "A0")
relevant_wells = [f"{row}{col+1}" for row in rows for col in cols]

In [5]:
double_volume = 230
volume = double_volume / 2 # volume to transfer (115 ul)

In [6]:
# the wells of the glycerol wellplate that actually contain glycerol
glycerol_wells = ["A1","A2","A3","A4","A5","A6","A7",]

# the amount of glycerol contained in each of the above wells 
# maximum defined by wellplate used (currently 2000)
glycerol_volume = 1800


# using a buffer when dispensing the glycerol yield higher precision in dispensed volume 
# defaults to False as this is not a task that requires very high precision
# for each aspiration the buffer_volume is additionally aspirated and after the transfers is blown out into the thrash
use_buffer = False
buffer_volume = p300.min_volume if use_buffer else 0

# only relevant if use_buffer
transfers_per_aspiration = p300.max_volume // volume

# number of target wells that can be filled with glycerol from a single glycerol_well
n_targets_per_g_well = int(glycerol_volume // (volume + buffer_volume/transfers_per_aspiration))

# check that enaugh glycerol is provided
assert n_targets_per_g_well * len(glycerol_wells) >= len(relevant_wells)

In [7]:
# Step 1 distribute glycerol

p300.pick_up_tip()

idx = 0 # tracking which wells in target already reacieved glycerol
for glycerol_well in glycerol_wells:
    # get slice of target wells that can be served from this well
    target_wells:list = relevant_wells[idx:(idx+n_targets_per_g_well)]
    if target_wells == []: break # all targets have been filled with glycerol

    p300.distribute(volume=volume,
                    source=glycerol_wellplate[glycerol_well],
                    dest=[target_wellplate[well] for well in target_wells],
                    disposal_volume=buffer_volume,
                    new_tip="never",)
    
    idx += n_targets_per_g_well

# drops tip into thrashbin at deckslot 12
p300.drop_tip(home_after=False) 


<InstrumentContext: p300_single_v1 in RIGHT>

In [8]:
# step 2 and 3

# number of times aspirating and dispensing into the same well for mixing
mix_repetitions = 2
# volume to aspirate and dispense
mix_volume = 1.5*volume # this is more or less arbitrary, could also even choose mix_volume < volume


p300.transfer(volume=volume,
              source=[culture_wellplate[well] for well in relevant_wells],
              dest=[target_wellplate[well] for well in relevant_wells],
              new_tip="always",
              mix_after=(mix_repetitions, mix_volume),
              )

<InstrumentContext: p300_single_v1 in RIGHT>

In [10]:
# if simulating you can print the commands with
for c in protocol.commands():
    print(c)